<a href="https://colab.research.google.com/github/BlackCurrantDS/DBSE_Project/blob/main/Part2_Rules_to_Feature_Vectors_BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook takes the association rules generated and results the feature vector of each rule. Also this generated co-relation matrix.

Input- 

input transaction file
Association rules files(the multiple temprory ones to make it run in parallel)

How to run:

Step1 :

Change the inputfilepath.

Change the folder path(if different from part1) in cell 1

Output-

This generates file item_relation.csv and miner.tmp.non_redundant_rules


In [84]:
inputfilepath = "/content/drive/MyDrive/Adult_DataSet1202/adult_train_transactions.csv"
file_path = "/content/drive/MyDrive/Adult_DataSet1202/Apriori1202/"


RuleFormatter

In [85]:
class RuleFormatter(object):
    
    @staticmethod
    def mydefaultLeft(item):
        return True
    
    @staticmethod
    def mydefaultRight(item):
        return True
    
    @staticmethod
    def mydefault(rule):
        #return True
        return len(rule.right_items) == 1#<= 2
    
    
    @staticmethod
    def massLeft(item):
        return item.isdigit()
    
    @staticmethod
    def massRight(item):
        return not item.isdigit()
    
    @staticmethod
    def mass(rule):
        return rule.lhs_string().isdigit() and (not rule.rhs_string().isdigit())
    
    @staticmethod
    def rna(rule):
        condition = (len(rule.right_items) == 1)
        condition &= ('rna_' in rule.rhs_string())
        condition &=  ('rna_' not in rule.lhs_string())
        return condition
    
    @staticmethod
    def tcrLeft(item):
        return item != 'CD4' and item != 'CD8'
    
    @staticmethod
    def tcrRight(item):
        return item == 'CD4' or item == 'CD8'    
    
    @staticmethod
    def tcr(rule):
        left_key = rule.lhs_string()
        right_key = rule.rhs_string()
        return ('CD4' not in left_key) and ('CD8' not in left_key) and (right_key == 'CD4' or right_key == 'CD8')
    
    @staticmethod
    def ank3Left(item):
        return item != 'CASE' and item != 'HEALTHY'
    
    @staticmethod
    def ank3Right(item):
        return item == 'CASE' or item == 'HEALTHY'
        
    @staticmethod
    def ank3(rule):
        left_key = rule.lhs_string()
        right_key = rule.rhs_string()
        return ('CASE' not in left_key) and ('HEALTHY' not in left_key) and (right_key == 'CASE' or right_key == 'HEALTHY')
    
    @staticmethod
    def spectLeft(item):
        return 'class@' not in item
    
    @staticmethod
    def spectRight(item):
        return 'class@' in item
        
    @staticmethod
    def spect(rule):
        flag = True
        for item in rule.right_items:
            if 'class@' not in item:
                flag = False
                break
        left_key = rule.lhs_string()
        return ('class@' not in left_key) and flag == True
    
    @staticmethod
    def kddLeft(item):
        return ('c_' in item) == False
    
    @staticmethod
    def kddRight(item):
        return 'c_' in item
        
    @staticmethod
    def kdd(rule):
        left_key = rule.lhs_string()
        right_key = rule.rhs_string()
        return ('c_' not in left_key) and (len(rule.right_items) == 1 and 'c_' in right_key)
    
    @staticmethod
    def tcrmLeft(item):
        return True
    
    @staticmethod
    def tcrmRight(item):
        return True
        
    @staticmethod
    def tcrm(rule):
        a_count1 = 0
        b_count1 = 0
        for item in rule.left_items:
            if 'b_' in item:
                b_count1 += 1
            if 'a_' in item:
                a_count1 += 1
        if a_count1 > 0 and b_count1 > 0: return False
        
        a_count2 = 0
        b_count2 = 0
        for item in rule.right_items:
            if 'b_' in item:
                b_count2 += 1
            if 'a_' in item:
                a_count2 += 1
        if a_count2 > 0 and b_count2 > 0: return False
        
        return (a_count1 > 0 and b_count2 > 0) or (b_count1 > 0 and a_count2 > 0)
    
    @staticmethod
    def ppiLeft(item):
        return True
    
    @staticmethod
    def ppiRight(item):
        return True
        
    @staticmethod
    def ppi(rule):
        a_count1 = 0
        b_count1 = 0
        for item in rule.left_items:
            if 'h@' in item:
                b_count1 += 1
            if 'v@' in item:
                a_count1 += 1
        if a_count1 > 0 and b_count1 > 0: return False
        
        a_count2 = 0
        b_count2 = 0
        for item in rule.right_items:
            if 'h@' in item:
                b_count2 += 1
            if 'v@' in item:
                a_count2 += 1
        if a_count2 > 0 and b_count2 > 0: return False
        
        return (a_count1 > 0 and b_count2 > 0) or (b_count1 > 0 and a_count2 > 0)
    
    
    @staticmethod
    def spliceLeft(item):
        return item != 'EI' and item != 'IE' and item != 'N@'
    
    @staticmethod
    def spliceRight(item):
        return item == 'D_0' or item == 'D_1' or item == 'N@'
        
    @staticmethod
    def splice(rule):
        left_key = rule.lhs_string()
        right_key = rule.rhs_string()
        return ('EI' not in left_key) and ('IE' not in left_key) and ('N@' not in left_key) and (right_key == 'EI' or right_key == 'IE' or right_key == 'N@')
        

Argument tuple

In [86]:


class ContrastParams(object):
    '''
    classdocs
    '''

    def __init__(self, delta1, delta2, share_threshold, n_lhs_features, n_rhs_features):
        '''
        Constructor
        '''
        self.delta1 = delta1 
        self.delta2 = delta2 
    
        self.share_threshold = share_threshold
        self.n_lhs_features = n_lhs_features
        self.n_rhs_features = n_rhs_features
        
class ARMParams(object):
    '''
    classdocs
    '''

    def __init__(self, minsup, minconf, itemset_max_size=-1):
        '''
        Constructor
        '''
        self.min_sup = minsup 
        self.min_conf = minconf
        self.itemset_max_size = itemset_max_size
        
class dbscanParams(object):
    
    def __init__(self, minpts, eps):
        self.min_pts = minpts
        self.eps = eps
        
class ARMFiles(object):
    '''
    classdocs
    '''

    def __init__(self, default_folder = file_path):
        '''
        Constructor
        '''
        self.temp_folder = default_folder
        
        self.itemset_tmp_file = self.temp_folder + 'miner.tmp.itemsets'
        self.rules_tmp_file = self.temp_folder + 'miner.tmp.rules'
        self.best_rules_file = self.temp_folder + 'miner.best_tmp.rules'
        
        self.interestingness_tmp_file = self.temp_folder +'miner.tmp.interestingness'
        self.probabilities_tmp_file = self.temp_folder +'miner.tmp.probabilities'
        
        self.feature_tmp_file = self.temp_folder +'miner.tmp.features'
        self.non_redundant_rule_tmp_file = self.temp_folder +'miner.tmp.non_redundant_rules'
        self.non_redundant_rule_feature_tmp_file = self.temp_folder + 'miner.tmp.non_redundant_rules.features'
        self.relation_tmp_file = self.temp_folder + 'relation_matrix.csv'
        
    def get_rule_file(self, i):
        return self.rules_tmp_file + '.' + str(i)
    

RuleMining helper

In [87]:

def string_2_itemset(key):
    if key == '':
        return []
    else: 
        return key.split(',')

def itemset_2_string(itemset):
    return ",".join(itemset)

def merge_itemsets(itemset_1, itemset_2):
    merged_items = []
    merged_items.extend(itemset_1)
    merged_items.extend(itemset_2)
    merged_items = list(set(merged_items))
    merged_items = sorted(merged_items)
    
    return merged_items

def get_full_path(prefix, file_name):
    if prefix == '': return file_name
    return prefix + '//' + file_name



Association Rule

In [88]:


class AssociationRule:
    def __init__(self, left, right):
        self.left_items = left
        self.right_items = right
        self.scores = []
        
    def length(self):
        return len(self.left_items) + len(self.right_items)
     
    def score(self, index):
        return self.scores[index]
    
    def lhs_string(self):
        return itemset_2_string(self.left_items)
        
    def rhs_string(self):
        return itemset_2_string(self.right_items)
    
    def serialize(self):
        left_key = self.lhs_string()
        right_key = self.rhs_string()
        return left_key + ">" + right_key
    
    @staticmethod        
    def string_2_rule(s):
        subStrings = s.split(">")
        left = string_2_itemset(subStrings[0].strip())
        right = string_2_itemset(subStrings[1].strip())
        return AssociationRule(left, right)

    def append_score(self, score):
        self.scores.append(score)
        
    def get_itemset(self):
        itemset = []
        itemset.extend(self.left_items)
        itemset.extend(self.right_items)
        itemset.sort()
        return itemset
        
        
    def rule_itemset_2_string(self):
        itemset = self.get_itemset()
        return itemset_2_string(itemset)
    
    def compute_basic_probs(self,frequent_itemsets, nTransactions):  
        
        left = frequent_itemsets[self.lhs_string()]
        right = frequent_itemsets[self.rhs_string()]
        
        both = frequent_itemsets[self.rule_itemset_2_string()]
        
        vector = {}
        
        ''' 1. P(A)'''
        p_A = left/nTransactions
        vector['A'] = p_A
        
        ''' 2. P(B)'''
        p_B = right/nTransactions
        vector['B'] = p_B
        
        ''' 3. P(~A)'''
        p_not_A = 1 - p_A
        vector['~A'] = p_not_A
        
        ''' 4. P(~B)'''
        p_not_B = 1 - p_B
        vector['~B'] = p_not_B
        
        ''' 5. P(AB) '''
        p_A_and_B = both/nTransactions
        vector['AB'] = p_A_and_B
        
        ''' 6. P(~AB)'''
        p_not_A_and_B = (right - both)/nTransactions
        vector['~AB'] = p_not_A_and_B
        
        ''' 7. P(A~B)'''
        p_A_and_not_B = (left - both)/nTransactions
        vector['A~B'] = p_A_and_not_B
        
        ''' 8. P(~A~B)'''
        p_not_A_and_not_B = 1 - (left + right - both)/nTransactions
        vector['~A~B'] = p_not_A_and_not_B 
        
        '''
        9. P(A|B)
        '''
        p_A_if_B = p_A_and_B / p_B
        vector['A|B'] = p_A_if_B
        
        '''
        10. P(~A|~B)
        '''
        p_not_A_if_not_B = p_not_A_and_not_B / p_not_B
        vector['~A|~B'] = p_not_A_if_not_B
        
        '''
        11. P(A|~B)
        '''
        p_A_if_not_B = p_A_and_not_B/p_not_B
        vector['A|~B'] = p_A_if_not_B
        
        '''
        12. p(~A|B)
        '''
        p_not_A_if_B = p_not_A_and_B / p_B
        vector['~A|B'] = p_not_A_if_B
        
        '''
        13. P(B|A)
        '''
        p_B_if_A = p_A_and_B / p_A
        vector['B|A'] = p_B_if_A
        
        '''
        14. P(~B|~A)
        '''
        p_not_B_if_not_A = p_not_A_and_not_B / p_not_A
        vector['~B|~A'] = p_not_B_if_not_A
        
        '''
        15. P(B|~A)
        '''
        p_B_if_not_A = p_not_A_and_B/p_not_A
        vector['B|~A'] = p_B_if_not_A
        
        '''
        16. p(~B|A)
        '''
        p_not_B_if_A = p_A_and_not_B / p_A
        vector['~B|A'] = p_not_B_if_A
        
        return vector
    
    def is_redundant_(self, bits, k, itemset, freq_itemset_dict): 
        '''
        Run out of items --> create rule and check format criterion
        '''
        if k >= len(itemset):
            items_1 = []
            items_2 = []
            for index in range(len(bits)):
                if bits[index] == True:
                    items_1.append(itemset[index])
                else:
                    items_2.append(itemset[index])
            for item in items_2:
                rule = AssociationRule(items_1, [item])
                confidence = freq_itemset_dict.getConfidence(rule)
                if confidence == 1: return True
            return False 
      
        value_domain = [True, False]
        for value in value_domain:
            bits[k] = value
            checker = self.is_redundant_(bits, k+1, itemset, freq_itemset_dict)
            if checker == True: return True
            bits[k] = True    
        return False
    
    '''
    Expand an item-set with equivalent items.
    '''
    def is_redundant(self, freq_itemset_dict):
        bits = [True for _ in self.left_items]
        checker = self.is_redundant_(bits, 0, self.left_items, freq_itemset_dict)
        if checker == True: return True
        
        bits =  [True for _ in self.right_items]
        return self.is_redundant_(bits, 0, self.right_items, freq_itemset_dict)
    
    '''
    Check if an item-set is satisfied condition of the rule. 
    '''
    def satisfy_rule(self, itemset, is_lhs = True):
        condition = self.left_items
        if is_lhs == False: condition = self.right_items
        if len(condition) > len(itemset) or len(itemset) == 0:
            return False
        for item in condition:
            if item not in itemset:
                return False
        return True
    

Getting realtion matrix

In [89]:


class RelationArray2D(object):
    '''
    classdocs
    '''

    def __init__(self, item_dict, relation_values):
        '''
        Constructor
        '''
        self.item_dict = item_dict
        self.relation_matrix = relation_values
        
        
    def get_value(self, item1, item2):
        i = self.item_dict[item1]
        j = self.item_dict[item2]
        return self.relation_matrix[i, j]
    
    def get_items(self):
        return self.item_dict.keys()
        
    def get_index(self, item):
        return self.item_dict[item]
    
class RelationArray1D(object):
    '''
    classdocs
    '''

    def __init__(self, item_dict, values):
        '''
        Constructor
        '''
        self.item_dict = item_dict
        self.values = values
        
        
    def get_value_at(self, index):
        return self.values[index]
    
    def get_items(self):
        return self.item_dict.keys()
        
    def get_value(self, item):
        return self.item_dict[item]

In [90]:

# Transaction databases, each transaction is a set of items
import numpy as np
from scipy import sparse
from scipy import stats
#from common.RelationArray import RelationArray2D
#from common.RelationArray import RelationArray1D

class DataSet:
    def __init__(self):
        self.current = 0
        self.train_data = []
        self.data_labels = []
        
    
    def __iter__(self):
        return iter(self.train_data)
                
    def size(self):
        return len(self.train_data)
    
    def get_transaction(self, index):
        return self.train_data[index]
    
    def clear(self):
        self.train_data.clear()
        
    def add_transaction(self, t):
        return self.train_data.append(t) 
        
    '''
    Load data set from a file. The input file must be formated in CSV (comma separated)
    class_index is used in the case of data-set with labels. 
    '''
    def load(self, file_path, class_index = -1, has_header = False):
        self.train_data = []
        if class_index != -1: self.data_labels = []
        
        with open(file_path, "r") as text_in_file:
            if has_header == True:
                text_in_file.readline()
                
            for line in text_in_file:
                #print("dataset script line", line)
                transaction = [x.strip() for x in line.split(',')]
                transaction = list(filter(None, transaction))
                #print("datset script transaction" , transaction)
                
                if (class_index != -1):
                    self.data_labels.append(transaction[class_index])
                    del transaction[class_index]
                
                self.train_data.append(list(set(transaction)))
            print("loading done")

    '''
    Return number of classes in data (if have).
    '''            
    def number_of_classes(self):
        if self.data_labels == None: return 0
        return len(set(self.data_labels))

    def convert_data_labels(self, inlier_name):
        Y_train = np.zeros(len(self.data_labels))
        for i in range(Y_train.shape[0]):
            if self.data_labels[i] == inlier_name:
                Y_train[i] = 1
            else: 
                Y_train[i] = -1
        return Y_train

    def convert_2_binary_format_with(self, items_dict, classes_dict = None):
        n_items = len(items_dict)
        X_train = np.zeros((self.size(), n_items))
        
        k = 0
        for transaction in self.train_data:
            for item in transaction:
                if item not in items_dict: 
                    print('not in features...')
                    continue
                i = items_dict[item]
                X_train[k, i] = 1.0
            k += 1
            
        Y_train = []
        if classes_dict is not None:
            for label in self.data_labels:
                if label not in classes_dict:
                    print('not in classes')
                    Y_train.append(-1)
                else:
                    Y_train.append(classes_dict[label]) 
        return X_train, np.array(Y_train)        
    
    def get_items_dict_(self):
        attr_dict = {}
        #check existing data
        for transaction in self.train_data:
            #print("get_items_dict_ transactions",transaction)
            for index in range (len(transaction)):
                item_name = transaction[index]
                if item_name not in attr_dict:
                    attr_dict[item_name] = True
        return attr_dict
    
    def get_class_list_(self):
        # Sort items and classes in alphabet order.
        return sorted(set(self.data_labels))
        
        

    '''
    Convert transaction data into binary format
    '''
    def convert_2_binary_format(self):
        
        attr_dict = self.get_items_dict_()
        print("convert_2_binary_format attr_dict", attr_dict)
      
         
        # Sort items and classes in alphabet order.
        classes_list = sorted(set(self.data_labels))
        print("classes_list", classes_list)
        items_list = sorted(attr_dict.keys())
        
        classes_dict = {classes_list[i] : i for i in range(len(classes_list))}
        print("classes_dict",classes_dict)
        attr_dict = {items_list[i] : i for i in range(len(items_list))}
        print("attr_dict",attr_dict)
        
        #Generate binary matrix (X_train) and array of labels(Y_train)
        X_train, Y_train = self.convert_2_binary_format_with(attr_dict, classes_dict)
                
        return RelationArray2D(attr_dict, sparse.csr_matrix(X_train)), RelationArray1D(classes_dict, np.array(Y_train))
        
    @staticmethod
    def write_relation_matrix_(matrix):
        with open(file_path+'item_relation.csv', 'w') as file_writer:
            item_names = sorted(matrix.item_dict.keys())
            print("item_names in static method",item_names)
            file_writer.write('o0o,')
            file_writer.write(','.join(item_names))
            file_writer.write('\n')
            for i in range(len(item_names)):
                print("in static method i ",i)
                file_writer.write(item_names[i] + ',')
                print(matrix.relation_matrix[i])
                file_writer.write(','.join(str(x) for x in matrix.relation_matrix[i].tolist()))
                file_writer.write('\n')
                
                
   
    '''
    This method estimates relationship among items. There're two kinds of relationship
    - Correlation:including negative correlation (<= -0.3) and positive correlation (>= 0.3)
    - Cover: threshold 1.0, including cover (2) and covered (-2) 
    '''
    def items_relationship(self):
        
        print ('Computing item relation matrix...')
        
        X_train, _ = self.convert_2_binary_format()
        
        print("X_train",X_train)
    
        correlation_matrix, p_values = stats.spearmanr(X_train.relation_matrix.todense(), axis = 0)

        print(correlation_matrix)
        
        #zeros_mask = (p_values <= 0.05).astype(int)
        #small_mask = (np.abs(correlation_matrix) >= 0.1).astype(int)
        
        #relation_matrix = correlation_matrix * small_mask * zeros_mask
        relation_matrix = correlation_matrix
        print(X_train.item_dict)
        print(relation_matrix)
        a = RelationArray2D(X_train.item_dict, relation_matrix)
        print(type(a))
        print("items_relationship a", a)
        DataSet.write_relation_matrix_(a)
        
        return a

Rule Miner

In [91]:

#from rules_mining.Apriori import Apriori
#from rules_mining.Generator import Generator
#from rules_mining.ItemsetDictionary import ItemsetDictionary

#from rules_mining.ItemsetFormatter import ItemsetFormatter
#from rules_mining.RuleFormatter import RuleFormatter
#from rules_mining.AssociationRule import AssociationRule

#from objective_measures.Interestingness import ObjectiveMeasure as om
#from rules_mining.RulesCollection import RulesDictionary

import json
import numpy as np
#from common.ArgumentTuple import ARMFiles

class RuleMiner(object):    
    '''
    This class is used to generate_itemsets_and_rules and store a Naive Belief System 
    by using the most confident association rules
    '''

    def __init__(self, filter_name, train_data_set):
        
        self.nthreads = 2
        self.files_info = ARMFiles(file_path)
        
        self.filter_name = filter_name
        self.data_set = train_data_set #taking input the dataset
        print("self.data_set is none or not", self.data_set)
        self.relation_matrix = None 
        if self.data_set is not None:
            print("Not none")
            self.relation_matrix = self.data_set.items_relationship() #calling
            print(self.data_set.items_relationship()) 
            print(type(self.relation_matrix))
            print("realation matrix",self.relation_matrix )
        
        
    '''
    Load generated frequent itemsets from file. 
    This method must be called after generate_frequent_itemsets is called
    '''
    def load_frequent_itemsets_as_dict(self):
        freq_itemset_dict = ItemsetDictionary(0)
        freq_itemset_dict.load_from_file(self.files_info.itemset_tmp_file)
        return freq_itemset_dict
    
    '''
    Load generated association rules from file. 
    This method must be called after generate_association_rules is called
    '''
    def load_rules_as_dict(self):
        rules_dict = RulesDictionary()
        
        for i in range(self.nthreads):
            rules_dict.load_from_file(self.files_info.get_rule_file(i))
                
        return rules_dict

        
    '''
    Generate frequent itemsets from data-set
    '''
    def generate_frequent_itemsets(self, arm_params):
        
        print ('generating frequent item-sets...')
        apriori = Apriori(self.data_set)
        apriori.generate_frequent_itemsets_vw(arm_params.min_sup * self.data_set.size(), 
                                              self.nthreads, 
                                              arm_params.itemset_max_size, 
                                              self.files_info.itemset_tmp_file)
        
    '''
    Generate association rules from data-set. 
    This method must be called after generate_frequent_itemsets(...) is called
    '''
    def generate_association_rules(self, arm_params):
        freq_itemsets_dict = self.load_frequent_itemsets_as_dict()
        
        print ('generating rules ....')
        itemset_formatter = getattr(ItemsetFormatter, self.filter_name)
        rule_formatter = getattr(RuleFormatter, self.filter_name)
        rule_generator = Generator(freq_itemsets_dict, 
                                   arm_params.min_conf, 
                                   itemset_formatter, 
                                   rule_formatter, 
                                   self.nthreads)
        rule_generator.execute(self.files_info.rules_tmp_file)
        
    '''
    Generate association rules and select K patterns with highest confidence.
    '''    
    def generate_itemsets_and_rules(self, arm_params):
        self.generate_frequent_itemsets(arm_params)
        self.generate_association_rules(arm_params)
        self.extract_features_4_all_rules()
         
    '''
    Compute confidence for all association rules generated from data-set
    '''
    def compute_confidence(self, association_rules_list):
        freq_itemset_dict = self.load_frequent_itemsets_as_dict()
        
        rule_confidence_dict = {}
        for rule in association_rules_list:
            left, _, both = freq_itemset_dict.get_frequency_combo(rule)
            rule_confidence_dict[rule.serialize()] = (both/left, both)
        return rule_confidence_dict

    '''
    Compute values of 31 interestingness measures for all association rules generated from data-set
    '''
    def compute_interestingness(self, output_file):
        print ('computing correlation among interestingness measures...')
        #measures = [om.confidence, om.lift]
        
        measures = [om.confidence, om.coverage, om.prevalence, om.recall, om.specificity, 
                    om.accuracy, om.lift, om.leverage, om.change_of_support, om.relative_risk, 
                    om.jaccard, om.certainty_factor, om.odd_ratio, om.yuleQ, om.yuleY, 
                    om.klosgen, om.conviction, om.weighting_dependancy, 
                    om.collective_strength, om.laplace_correction, om.jmeasure, 
                    om.one_way_support, om.two_way_support, om.two_ways_support_variation, 
                    om.linear_correlation_coefficient, om.piatetsky_shapiro, om.loevinger,
                    om.information_gain, om.sebag_schoenauner, om.least_contradiction, 
                    om.odd_multiplier, om.counter_example_rate, om.zhang]
        
        print('loading frequent item-sets....')
        freq_itemsets_dict =  self.load_frequent_itemsets_as_dict()
        association_rules = self.load_association_rules()
        
        print ('computing interestingness for all rules ....')
        
        with open(output_file, 'w') as write_file:
            total = freq_itemsets_dict.ntransactions
            for rule in association_rules:
                left, right, both = freq_itemsets_dict.get_frequency_combo(rule)
                interestingness = []
                for index in range(len(measures)):
                    value = measures[index](left, right, both, total)
                    interestingness.append(value)
                write_file.write(rule.serialize() + ';')            
                write_file.write(';'.join([str(x) for x in interestingness]))
                write_file.write('\n')
                
    '''
    Determine collection of features for LHS and RHS of rules.
    This method returns two dictionaries for LHS and RHS respectively. 
    Each entry of the dictionaries are (the name of item : its index in feature vector)
    ''' 
    def _get_feature_names(self):
        
        left_features = []
        right_features = []
        
        left_filter = getattr(RuleFormatter, self.filter_name + 'Left')
        print("_get_feature_names left_filter ",left_filter) #this gives memory location
        right_filter = getattr(RuleFormatter, self.filter_name + 'Right')
        print("_get_feature_names right_filter",right_filter) #this gives memory location
        for item in self.relation_matrix.get_items():
            #print("_get_feature_names item ",item) #gives item one by one a1@30-39
            #print("_get_feature_names left_filter(item)",left_filter(item))
            if left_filter(item):
                left_features.append(item) #true or false
            if right_filter(item):
                right_features.append(item) 
        print("_get_feature_names",sorted(left_features)) #list of all the items wihtout class lables
        print("_get_feature_names",sorted(right_features))   #list of items with class labels  
        return sorted(left_features), sorted(right_features)
    
    '''
    Extract feature for an item-set.
    '''
    def _extract_features_4_itemset(self, itemset, feature_names):
        n = len(feature_names)
        f_vector = [0 for _ in range(n)]
        for item in itemset:
            print("_extract_features_4_itemset",item)
            for i in range(n):
                print("_extract_features_4_itemset i ", i)
                a = self.relation_matrix.get_value(item, feature_names[i])
                print("_extract_features_4_itemset a", a)
                if (abs(f_vector[i]) < abs(a)):
                    f_vector[i] = a
      
        return np.array(f_vector) 
    '''
    Extract feature vectors for all rules 
    '''
    def extract_features_4_all_rules(self):
        left_features, right_features  = self._get_feature_names()
        left_count = len(left_features)
        right_count = len(right_features)
        
        print('Write number of features for LHS and RHS')
        features_writer = open(self.files_info.non_redundant_rule_tmp_file, 'w')
        features_writer.write(str(left_count))
        features_writer.write('\n')
        features_writer.write(str(right_count))
        features_writer.write('\n')
        
        print('Starting extraction...')
        for i in range(self.nthreads):
            input_file = self.files_info.get_rule_file(i)
            
            with open(input_file, 'r') as rules_reader:
                for line in rules_reader:
                    print("line",line) #for each rule a3@0-4>class@no
                    rule = AssociationRule.string_2_rule(line.strip())
                    print("string to rule",rule) #memory location from class
                    a = self._extract_features_4_itemset(rule.left_items, left_features)
                    print("a",a) 
                    b = self._extract_features_4_itemset(rule.right_items, right_features)
                    print("b",b)
                    f_vector = np.concatenate((a, b))
                    print("f_vector",f_vector)
                    '''
                    Write a feature vector to file
                    '''               
                    features_writer.write(json.dumps((rule.serialize(),f_vector.tolist())))
                    features_writer.write('\n')
        features_writer.close()
        
    '''
    Load feature vectors for all non-redundant rules
    '''
    def load_feature_vectors(self):
        data = []
        lengths = []
        
        with open(self.files_info.non_redundant_rule_tmp_file, 'r') as feature_reader:
            print('Loading number of LHS and RHS features...')
            lhs_count = int(feature_reader.readline())
            rhs_count = int(feature_reader.readline())
            print('Loading feature vectors... ')
            for line in feature_reader:
                rule_text, f_vector = json.loads(line.strip())
                rule = AssociationRule.string_2_rule(rule_text.strip())
                lengths.append(rule.length())
                data.append(f_vector)
                
                
        return np.array(data), lengths, lhs_count, rhs_count
        
    '''
    Load non-redundant rules from a file.
    '''
    def load_association_rules(self):
        association_rules_list = []
        with open(self.files_info.non_redundant_rule_tmp_file, 'r') as rules_reader:
            rules_reader.readline()
            rules_reader.readline()
            
            for line in rules_reader:
                rule_text, _ = json.loads(line.strip())
                association_rules_list.append(AssociationRule.string_2_rule(rule_text.strip()))
        return association_rules_list
        
    '''
    Load non-redundant rules and their feature vectors from a file
    '''
    def load_rules_and_features(self):
        rules_and_their_features = {}
        with open(self.files_info.non_redundant_rule_tmp_file, 'r') as feature_reader:
            lhs_count = int(feature_reader.readline())
            rhs_count = int(feature_reader.readline())
            
            for line in feature_reader:
                rule_text, f_vector = json.loads(line.strip())
                rules_and_their_features[rule_text] = f_vector
        return rules_and_their_features, lhs_count, rhs_count
        
    

In [92]:
train_data_set = DataSet()

In [93]:
train_data_set.load(inputfilepath)

loading done


In [94]:
ruleminer = RuleMiner('spect',train_data_set)

Streaming output truncated to the last 5000 lines.
  1.74796487e-02  8.22318879e-03  2.40605105e-03  5.73240825e-07
 -1.39401071e-02 -9.50264581e-04  2.69148852e-03 -1.83260890e-02
 -3.87325393e-03  6.82140124e-03 -1.16118509e-03  5.23319879e-02
  4.40553678e-02 -8.14999726e-03 -1.04831008e-02 -8.34356477e-03
 -2.99304215e-03 -9.78849500e-03  3.72026017e-03 -1.53189396e-02
 -8.70303832e-03  4.66996511e-02  6.62805112e-03 -2.63186357e-04
 -5.61212662e-03 -1.03755951e-02 -1.67868439e-02  1.85025099e-02
  1.59635854e-02 -7.35874350e-05  7.35874350e-05 -2.08380402e-02
  2.08380402e-02]
in static method i  49
[-3.46518701e-03  4.25779918e-04  5.92123861e-03 -5.30247523e-03
  8.29655565e-03  1.60269742e-03  3.00432433e-03 -3.85889255e-03
  1.05182998e-02 -6.82843949e-03  8.60686659e-03 -6.52443495e-04
 -4.12622487e-04 -3.68795893e-03  2.67201192e-03 -3.35576665e-03
 -1.35778422e-03  3.24157141e-03 -4.21109079e-03 -1.09189772e-03
 -7.14727016e-04 -5.05365035e-04 -1.20326814e-03 -1.33743996e-0

In [95]:
get_feature_names = ruleminer._get_feature_names()

_get_feature_names left_filter  <function RuleFormatter.spectLeft at 0x7f60d0506f28>
_get_feature_names right_filter <function RuleFormatter.spectRight at 0x7f60d05069d8>
_get_feature_names ['a0@24.3_31.6', 'a0@31.6_38.9', 'a0@38.9_46.2', 'a0@46.2_53.5', 'a0@53.5_60.8', 'a0@60.8_68.1', 'a0@68.1_75.4', 'a0@75.4_82.7', 'a0@82.7g', 'a0@l24.3', 'a10@19999.8_29999.7', 'a10@29999.7_39999.6', 'a10@39999.6_49999.5', 'a10@89999.1g', 'a10@9999.9_19999.8', 'a10@l9999.9', 'a11@1306.8_1742.4', 'a11@1742.4_2178', 'a11@2178_2613.6', 'a11@2613.6_3049.2', 'a11@3484.8_3920.4', 'a11@3920.4g', 'a11@435.6_871.2', 'a11@871.2_1306.8', 'a11@l435.6', 'a12@10.8_20.6', 'a12@20.6_30.4', 'a12@30.4_40.2', 'a12@40.2_50', 'a12@50_59.8', 'a12@59.8_69.6', 'a12@69.6_79.4', 'a12@79.4_89.2', 'a12@89.2g', 'a12@l10.8', 'a13@?', 'a13@Hong', 'a13@Iran', 'a13@Italy', 'a13@Laos', 'a13@Peru', 'a13@cam', 'a13@can', 'a13@china', 'a13@col', 'a13@cuba', 'a13@domi', 'a13@ecu', 'a13@elsa', 'a13@eng', 'a13@fran', 'a13@ger', 'a13@gree',

In [96]:
extract_features_4_all_rules = ruleminer.extract_features_4_all_rules()

Streaming output truncated to the last 5000 lines.
_extract_features_4_itemset i  48
_extract_features_4_itemset a 0.0074631643739331305
_extract_features_4_itemset i  49
_extract_features_4_itemset a 0.0004681035130805398
_extract_features_4_itemset i  50
_extract_features_4_itemset a 0.0065794941671574285
_extract_features_4_itemset i  51
_extract_features_4_itemset a 0.0007786346474138424
_extract_features_4_itemset i  52
_extract_features_4_itemset a -0.01301575961284925
_extract_features_4_itemset i  53
_extract_features_4_itemset a 0.00648012323811701
_extract_features_4_itemset i  54
_extract_features_4_itemset a 0.00015026958439615508
_extract_features_4_itemset i  55
_extract_features_4_itemset a -0.0029108389428967197
_extract_features_4_itemset i  56
_extract_features_4_itemset a -0.0029108389428967197
_extract_features_4_itemset i  57
_extract_features_4_itemset a -0.006255658599781259
_extract_features_4_itemset i  58
_extract_features_4_itemset a 0.0006004535571591217
_ex